# Sentiment Analysis on IMDB Movie Reviews using LSTM

[Long short-term memory (LSTM)](https://en.wikipedia.org/wiki/Long_short-term_memory) is a recurrent neural network (RNN) architecture that remembers values over arbitrary intervals. Stored values are not modified as learning proceeds. RNNs allow forward and backward connections between neurons.

An LSTM network contains LSTM units instead of, or in addition to, other network units. An LSTM unit remembers values for either long or short time periods. The key to this ability is that it uses no activation function within its recurrent components. Thus, the stored value is not iteratively modified and the gradient does not tend to vanish when trained with backpropagation through time.

- 0 - negative

- 1 - negative

## Load & Read Datasets

Trains an LSTM model on the IMDB sentiment classification task. The dataset is actually too small for LSTM to be of any advantage compared to simpler, much faster methods such as TF-IDF + LogReg.

**Notes:**
- RNNs are tricky. Choice of batch size is important, choice of loss and optimizer is critical, etc. Some configurations won't converge.
- LSTM loss decrease patterns during training can be quite different from what you see with CNNs/MLPs/etc.

In [48]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
import numpy as np

max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 200

Dataset of 25,000 movies reviews from IMDB, labeled by sentiment (positive/negative). Reviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers). For convenience, words are indexed by overall frequency in the dataset, so that for instance the integer "3" encodes the 3rd most frequent word in the data. This allows for quick filtering operations such as: "only consider the top 10,000 most common words, but eliminate the top 20 most common words".

**Arguments:**

- **path:** if you do not have the data locally (at '~/.keras/datasets/' + path), it will be downloaded to this location.

- **num_words:** integer or None. Top most frequent words to consider. Any less frequent word will appear as oov_char value in the sequence data.

- **skip_top:** integer. Top most frequent words to ignore (they will appear as oov_char value in the sequence data).

- **maxlen:** int. Maximum sequence length. Any longer sequence will be truncated.

- **seed:** int. Seed for reproducible data shuffling.

- **start_char:** int. The start of a sequence will be marked with this character. Set to 1 because 0 is usually the padding character.

- **oov_char:** int. words that were cut out because of the num_words or skip_top limit will be replaced with this character.

- **index_from:** int. Index actual words with this index and higher.

*Source: https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification*

In [ ]:
'''
(x_train, y_train), (x_test, y_test) = imdb.load_data(path="imdb.npz",
                                                      num_words=None,
                                                      skip_top=0,
                                                      maxlen=None,
                                                      seed=113,
                                                      start_char=1,
                                                      oov_char=2,
                                                      index_from=3)
'''

In [38]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

In [41]:
print (X_train.shape, X_test.shape)

(25000,) (25000,)


In [21]:
X_train

array([ list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]),
       list([1, 194, 1153, 194, 8

In [57]:
for i in range(0,3):
    print (len(X_train[i]))

218
189
141


Making each training data sample of same size/length.

In [58]:
print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

Pad sequences (samples x time)


In [23]:
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

X_train shape: (25000, 80)
X_test shape: (25000, 80)


In [24]:
X_train

array([[   15,   256,     4, ...,    19,   178,    32],
       [  125,    68,     2, ...,    16,   145,    95],
       [  645,   662,     8, ...,     7,   129,   113],
       ..., 
       [  529,   443, 17793, ...,     4,  3586,     2],
       [  286,  1814,    23, ...,    12,     9,    23],
       [   97,    90,    35, ...,   204,   131,     9]], dtype=int32)

In [60]:
for i in range(0,3):
    print (len(X_train[i]))

80
80
80


In [25]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2)) # 128 = dimensionality of the output space
model.add(Dense(1, activation='sigmoid')) 

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Build model...


In [31]:
print('Train...')
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=3,
          validation_data=(X_test, y_test))

Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 530s - loss: 0.3970 - acc: 0.8244 - val_loss: 0.3701 - val_acc: 0.8367
Epoch 2/3
25000/25000 [==============================] - 570s - loss: 0.2797 - acc: 0.8908 - val_loss: 0.3918 - val_acc: 0.8282
Epoch 3/3
25000/25000 [==============================] - 532s - loss: 0.2250 - acc: 0.9128 - val_loss: 0.4230 - val_acc: 0.8292


In [32]:
score, acc = model.evaluate(X_test, y_test, batch_size=batch_size)

print('Test score:', score)
print('Test accuracy:', acc)

25000/25000 [==============================] - 152s   
Test score: 0.422962269306
Test accuracy: 0.829199996471


## Reference

https://github.com/fchollet/keras/blob/master/examples/imdb_lstm.py